In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()
output_null = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./kit_part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error, output_null
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://www.rockauto.com/en/parts/' + input_.loc[a, 'Kit_Manufacturer'].replace('+', '%2B').replace(',', '%2C') + ',' + input_.loc[a, 'Kit_Part_Number'] + ',Lennon,' + input_.loc[a, 'Kit_Type_Code']

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =

                no_parts_found = html.xpath('//span[@class="navlabellink nvoffset nnormal"]/text()')

                if len(no_parts_found) != 0 and no_parts_found[0].strip() == 'No Parts Found':
                    crawler_status = 'null'
                    break
                    
                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_json = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_supplemental")]/@value')[0]) for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_json_2 = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_essential")]/@value')[0]) for part in list_part]

                # = = = = = = = = = = = = = = =

                index_part = -1
                for i in range(len(list_json)):
                    if list_json[i]['catalogname'].strip() == input_.loc[a, 'Kit_Manufacturer'] and list_json[i]['partnumber'].strip() == input_.loc[a, 'Kit_Part_Number'] and list_json_2[i]['parttype'].strip() == input_.loc[a, 'Kit_Type_Code']:
                        if index_part != -1:
                            continue
                        index_part = i

                # = = = = = = = = = = = = = = =

                category = html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                type_ = html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_note_1 = list_part[index_part].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                note_1 = '' if len(list_note_1) == 0 else ';'.join([note_1.strip() for note_1 in list_note_1])

                # = = = = = = = = = = = = = = =
                
                list_note_2 = list_part[index_part].xpath('./descendant::span[contains(@title, "Please make sure")]/span/text()')
                note_2 = '' if len(list_note_2) == 0 else ';'.join([note_2.strip() for note_2 in list_note_2])

                # = = = = = = = = = = = = = = =

                src = list_part[index_part].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                src = '' if len(src) == 0 else 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')

                # = = = = = = = = = = = = = = =
                
                oe = list_part[index_part].xpath('./descendant::span[contains(@title, "Replaces these Alternate")]')
                oe = '' if len(oe) == 0 else oe[0].xpath('./text()')[0].strip().replace(', ', ';')
                oe = ';'.join(sorted(oe.split(';')))
                
                # = = = = = = = = = = = = = = =
                
                part_key = list_json_2[index_part]['partkey'].strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                                         'Kit_序号': int(input_.loc[a, 'Kit_No.']),
                                         'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                                         'Kit_Note': input_.loc[a, 'Kit_Note'],
                                         'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                         'Kit_Category': category,
                                         'Kit_Type': type_,
                                         'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                         'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number'],
                                         'Kit_Vehcile': '',
                                         'Kit_OE': oe,
                                         'Kit_Note_1': note_1,
                                         'Kit_Note_2': note_2,
                                         'Kit_Pic': '',
                                         'Kit_Url': url,
                                         'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + part_key,
                                         'Kit_Src': src,
                                         'Kit_Part_Key': part_key,
                                         'Kit_JOIN_TCPK': input_.loc[a, 'Kit_Type_Code']+';'+part_key}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                                     'Kit_No.': int(input_.loc[a, 'Kit_No.']),
                                     'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                                     'Kit_Note': input_.loc[a, 'Kit_Note'],
                                     'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                     'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                     'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'null':
            df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                                     'Kit_序号': int(input_.loc[a, 'Kit_No.']),
                                     'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                                     'Kit_Note': input_.loc[a, 'Kit_Note'],
                                     'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                     'Kit_Category': '',
                                     'Kit_Type': '',
                                     'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                     'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number'],
                                     'Kit_Vehcile': '',
                                     'Kit_OE': '',
                                     'Kit_Note_1': '',
                                     'Kit_Note_2': '',
                                     'Kit_Pic': '',
                                     'Kit_Url': '',
                                     'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_Src': '',
                                     'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_JOIN_TCPK': input_.loc[a, 'Kit_Type_Code']+';'+input_.loc[a, 'Kit_Part_Key']}])

            output_null = pd.concat([output_null, df_temp], ignore_index=True).fillna('')
    
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Info'] + ' - ' + input_.loc[a, 'Kit_No.'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./kit_part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info', 'Kit_No.'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./kit_part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
if len(output_null) != 0:
    output_null = output_null.drop_duplicates(ignore_index=True)
    output_null = output_null.sort_values(by=['Info', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
    output_null.to_excel('./kit_part_null-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
    print('爬虫存在null')
    print()
print('Done ~')

总数量：1976

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 3
[尝试次数：1] - [剩余数量：1964] - [当前时间：14:34:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 2
[尝试次数：1] - [剩余数量：1963] - [当前时间：14:34:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10195072 - 1
[尝试次数：1] - [剩余数量：1962] - [当前时间：14:34:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110004 - 1
[尝试次数：1] - [剩余数量：1961] - [当前时间：14:34:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 5
[尝试次数：1] - [剩余数量：1960] - [当前时间：14:34:06]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 1
[尝试次数：1] - [剩余数量：1959] - [当前时间：14:34:06]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 8
[尝试次数：1] - [剩余数量：1958] - [当前时间：14:34:06]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10195072 - 5
[尝试次数：1] - [剩余数量：1957] - [当前时间：14:34:08]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10195072 - 4
[尝试次数：

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454520 - 6
[尝试次数：1] - [剩余数量：1892] - [当前时间：14:34:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10519376 - 6
[尝试次数：1] - [剩余数量：1891] - [当前时间：14:34:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454508 - 2
[尝试次数：1] - [剩余数量：1890] - [当前时间：14:34:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10519376 - 2
[尝试次数：2] - [剩余数量：1889] - [当前时间：14:34:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10519380 - 1
[尝试次数：1] - [剩余数量：1888] - [当前时间：14:34:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454520 - 3
[尝试次数：2] - [剩余数量：1887] - [当前时间：14:34:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10519380 - 2
[尝试次数：1] - [剩余数量：1886] - [当前时间：14:34:41]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454512 - 9
[尝试次数：2] - [剩余数量：1885] - [当前时间：14:34:43]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10519380 - 5
[尝试次数：1] - [剩余数量

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890260 - 9
[尝试次数：3] - [剩余数量：1819] - [当前时间：14:35:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890300 - 8
[尝试次数：1] - [剩余数量：1818] - [当前时间：14:35:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890300 - 7
[尝试次数：1] - [剩余数量：1817] - [当前时间：14:35:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890296 - 4
[尝试次数：3] - [剩余数量：1816] - [当前时间：14:35:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10981372 - 1
[尝试次数：1] - [剩余数量：1815] - [当前时间：14:35:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10981372 - 2
[尝试次数：1] - [剩余数量：1814] - [当前时间：14:35:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890288 - 8
[尝试次数：3] - [剩余数量：1813] - [当前时间：14:35:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10981372 - 3
[尝试次数：1] - [剩余数量：1812] - [当前时间：14:35:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10981372 - 5
[尝试次数：1] - [剩余数量

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481 - 2
[尝试次数：1] - [剩余数量：1745] - [当前时间：14:35:49]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481 - 1
[尝试次数：1] - [剩余数量：1744] - [当前时间：14:35:50]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158089 - 3
[尝试次数：2] - [剩余数量：1743] - [当前时间：14:35:51]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481 - 8
[尝试次数：1] - [剩余数量：1742] - [当前时间：14:35:51]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481 - 5
[尝试次数：2] - [剩余数量：1741] - [当前时间：14:35:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10981384 - 4
[尝试次数：3] - [剩余数量：1740] - [当前时间：14:35:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481 - 9
[尝试次数：1] - [剩余数量：1739] - [当前时间：14:35:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481 - 4
[尝试次数：3] - [剩余数量：1738] - [当前时间：14:35:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481 - 6
[尝试次数：1] - [剩余数量

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804013 - 6
[尝试次数：2] - [剩余数量：1673] - [当前时间：14:36:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804017 - 1
[尝试次数：1] - [剩余数量：1672] - [当前时间：14:36:41]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804017 - 5
[尝试次数：1] - [剩余数量：1671] - [当前时间：14:36:41]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804013 - 4
[尝试次数：1] - [剩余数量：1670] - [当前时间：14:36:41]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804013 - 7
[尝试次数：1] - [剩余数量：1669] - [当前时间：14:36:41]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804017 - 4
[尝试次数：1] - [剩余数量：1668] - [当前时间：14:36:42]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804049 - 1
[尝试次数：1] - [剩余数量：1667] - [当前时间：14:36:43]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804017 - 3
[尝试次数：1] - [剩余数量：1666] - [当前时间：14:36:43]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804049 - 3
[尝试次数：1] - [剩余数量

...

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305544 - 8
[尝试次数：1] - [剩余数量：285] - [当前时间：14:49:34]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305588 - 3
[尝试次数：1] - [剩余数量：284] - [当前时间：14:49:36]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305588 - 2
[尝试次数：1] - [剩余数量：283] - [当前时间：14:49:36]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305580 - 2
[尝试次数：2] - [剩余数量：282] - [当前时间：14:49:36]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305492 - 2
[尝试次数：3] - [剩余数量：281] - [当前时间：14:49:37]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305544 - 9
[尝试次数：4] - [剩余数量：280] - [当前时间：14:49:37]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305580 - 3
[尝试次数：2] - [剩余数量：279] - [当前时间：14:49:37]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305608 - 1
[尝试次数：1] - [剩余数量：278] - [当前时间：14:49:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305600 - 3
[尝试次数：1] - [剩余数量：277] - [当前时间：14:

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305776 - 9
[尝试次数：2] - [剩余数量：211] - [当前时间：14:50:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305792 - 5
[尝试次数：1] - [剩余数量：210] - [当前时间：14:50:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305792 - 8
[尝试次数：1] - [剩余数量：209] - [当前时间：14:50:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305792 - 9
[尝试次数：1] - [剩余数量：208] - [当前时间：14:50:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305792 - 10
[尝试次数：1] - [剩余数量：207] - [当前时间：14:50:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305800 - 2
[尝试次数：1] - [剩余数量：206] - [当前时间：14:50:17]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305800 - 4
[尝试次数：1] - [剩余数量：205] - [当前时间：14:50:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305784 - 5
[尝试次数：1] - [剩余数量：204] - [当前时间：14:50:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305800 - 5
[尝试次数：1] - [剩余数量：203] - [当前时间：14

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527948 - 2
[尝试次数：1] - [剩余数量：137] - [当前时间：14:51:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305824 - 10
[尝试次数：3] - [剩余数量：136] - [当前时间：14:51:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527948 - 3
[尝试次数：1] - [剩余数量：135] - [当前时间：14:51:03]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527952 - 2
[尝试次数：1] - [剩余数量：134] - [当前时间：14:51:04]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306204 - 6
[尝试次数：4] - [剩余数量：133] - [当前时间：14:51:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527952 - 3
[尝试次数：1] - [剩余数量：132] - [当前时间：14:51:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527952 - 4
[尝试次数：1] - [剩余数量：131] - [当前时间：14:51:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527952 - 1
[尝试次数：1] - [剩余数量：130] - [当前时间：14:51:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527952 - 5
[尝试次数：1] - [剩余数量：129] - [当前时间：14

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921200 - 6
[尝试次数：3] - [剩余数量：63] - [当前时间：14:51:42]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984 - 6
[尝试次数：1] - [剩余数量：62] - [当前时间：14:51:44]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984 - 7
[尝试次数：1] - [剩余数量：61] - [当前时间：14:51:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921228 - 7
[尝试次数：2] - [剩余数量：60] - [当前时间：14:51:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984 - 9
[尝试次数：1] - [剩余数量：59] - [当前时间：14:51:46]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658980 - 3
[尝试次数：2] - [剩余数量：58] - [当前时间：14:51:46]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984 - 4
[尝试次数：3] - [剩余数量：57] - [当前时间：14:51:47]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984 - 8
[尝试次数：1] - [剩余数量：56] - [当前时间：14:51:47]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921228 - 8
[尝试次数：3] - [剩余数量：55] - [当前时间：14:51:47]

[

[null] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9854500 - 2
[尝试次数：2] - [剩余数量：0] - [当前时间：14:52:30]

输出ing...

爬虫存在null

Done ~
